In [8]:
import matplotlib.pyplot as plt
import cv2, os
from function.converse_label import get_label
from PIL import Image
from keras.preprocessing.image import img_to_array
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms, utils
import matplotlib.pyplot as plt
train_data = []
train_labels = []
label, category = get_label('labelByHand.csv')
print(label)
print(category)
path = 'data'
for i, folderName in enumerate(os.listdir(path)):
    print("folder:", folderName)
    if folderName[0] != ".":
        for file_name in os.listdir(os.path.join(path, folderName)):
            if file_name == "train":
                for image in os.listdir(os.path.join(path, folderName, file_name)):
                    if image[0] != '.':
                        img = cv2.imread(os.path.join(path, folderName, file_name, image))
                        #plt.imshow(img)
                        img = cv2.resize(img, (64, 64))
                        transform = transforms.ToTensor()
                        tensor = transform(img)
                        train_data.append(tensor)
                        train_labels.append(label[i])
print("end")

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 10, 11, 12, 12, 13, 14, 15, 14, 16, 17, 1, 18, 19, 2, 20, 14, 21, 8, 20, 22, 22, 22, 23, 24, 25, 26, 20, 27, 28, 29, 30]
31
folder: 01
folder: 02
folder: 03
folder: 04
folder: 05
folder: 06
folder: 07
folder: 08
folder: 09
folder: 10
folder: 11
folder: 12
folder: 13
folder: 14
folder: 15
folder: 16
folder: 17
folder: 18
folder: 19
folder: 20
folder: 21
folder: 22
folder: 23
folder: 24
folder: 25
folder: 26
folder: 27
folder: 28
folder: 29
folder: 30
folder: 31
folder: 32
folder: 33
folder: 34
folder: 35
folder: 36
folder: 37
folder: 38
folder: 39
folder: 40
folder: 41
folder: 42
end


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, train_labels, test_size = 0.2, shuffle = True)
print("訓練資料集數量：", len(x_train))
print("測試資料集數量：", len(x_test))
batch_size = 256

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = torch.LongTensor(label)

    def __getitem__(self,index):
        return self.data[index],self.label[index]

    def __len__(self):
        return len(self.data)
trainDataset = MyDataset(x_train, y_train)
testDataset = MyDataset(x_test, y_test)

訓練資料集數量： 10078
測試資料集數量： 2520


In [10]:
batch_size = 128
# 建立 DataLoader
train_loader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(testDataset, batch_size=batch_size)
print(len(test_loader))

# 測試以 DataLoader 載入資料
for X, y in test_loader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y)

20
Shape of X [N, C, H, W]:  torch.Size([128, 3, 64, 64])
Shape of y:  torch.Size([128]) tensor([ 1, 17, 22, 20,  2, 22,  1, 22, 30, 20, 14, 14, 14,  2, 14,  4, 10, 15,
        14,  8, 29, 30,  5,  1,  8, 27, 30, 22, 12, 11, 25, 20, 16, 20, 14,  0,
        26, 10,  2, 12, 22, 27, 18,  7,  7,  9, 20,  7,  1,  8,  2, 14, 20,  1,
        13,  1, 14, 17,  1, 20,  8, 21, 28,  1,  8,  3, 19, 12, 15,  8,  4, 11,
         8,  3, 29, 12,  2, 22,  5,  9,  5,  2, 15, 14, 15, 24, 24,  7,  1, 22,
        19, 14, 26,  5,  1, 14, 17, 22, 19, 21, 22,  9, 11, 27,  1,  3, 18,  9,
        18,  8, 19,  7,  8, 23, 16, 29, 28, 12,  0,  2, 21, 21, 12, 20,  1, 26,
        30, 23])
Shape of X [N, C, H, W]:  torch.Size([128, 3, 64, 64])
Shape of y:  torch.Size([128]) tensor([28,  0, 12, 30,  6,  4, 24,  9, 10, 10,  9, 20, 16, 20, 20, 12,  3, 25,
        29,  1, 11,  2, 15, 27, 21, 18, 19, 22,  1, 12,  2,  6, 14, 23, 17, 17,
        11, 29, 15, 20, 22,  3, 26, 12,  2, 21, 22, 30, 29,  1,  9, 18,  8, 27,
        

In [11]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        # 載入 ResNet18 類神經網路結構
        self.model = models.resnet18(pretrained=True)
        # 修改輸出層輸出數量
        self.model.fc = nn.Linear(512, 31)

    def forward(self, x):
        logits = self.model(x)
        return logits
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model = ResNet18().to(device)

Using cpu device


In [12]:
loss_fn = nn.CrossEntropyLoss()
# 學習優化器
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    # 資料總筆數
    size = len(dataloader.dataset)
    # 將模型設定為訓練模式
    model.train()
    # 批次讀取資料進行訓練
    for batch, (X, y) in enumerate(dataloader):
        # 將資料放置於 GPU 或 CPU
        X, y = X.to(device), y.to(device)

        pred = model(X)         # 計算預測值
        loss = loss_fn(pred, y) # 計算損失值（loss）

        optimizer.zero_grad()   # 重設參數梯度（gradient）
        loss.backward()         # 反向傳播（backpropagation）
        optimizer.step()        # 更新參數
        # print
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    # data_size
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # 將模型設定為驗證模式
    model.eval()
    # initial
    test_loss, correct = 0, 0
    # 驗證模型準確度
    with torch.no_grad():  # 不要計算參數梯度
        for X, y in dataloader:
            # 將資料放置於 GPU 或 CPU
            X, y = X.to(device), y.to(device)
            # 計算預測值
            pred = model(X)
            # 計算損失值的加總值
            test_loss += loss_fn(pred, y).item()
            # 計算預測正確數量的加總值
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # 計算平均損失值與正確率
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 6

# 開始訓練模型
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("end")

Epoch 1
-------------------------------
loss: 3.996875  [    0/10078]
Test Error: 
 Accuracy: 96.5%, Avg loss: 0.305282 

Epoch 2
-------------------------------
loss: 0.294372  [    0/10078]
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.122706 

Epoch 3
-------------------------------
loss: 0.086628  [    0/10078]
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.077807 

Epoch 4
-------------------------------
loss: 0.042399  [    0/10078]
Test Error: 
 Accuracy: 99.3%, Avg loss: 0.058598 

Epoch 5
-------------------------------
loss: 0.026849  [    0/10078]
Test Error: 
 Accuracy: 99.3%, Avg loss: 0.048153 

Epoch 6
-------------------------------
loss: 0.019310  [    0/10078]
Test Error: 
 Accuracy: 99.4%, Avg loss: 0.041530 

end


In [15]:
torch.save(model.state_dict(), 'model/resNet/ResNet_3.pt')
torch.save(model, 'model/resNet/ResNet_new3.pt')


In [28]:
num_classes = 31
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        # 載入 ResNet18 類神經網路結構
        self.model = models.resnet18(pretrained=True)
        # 修改輸出層輸出數量
        self.model.fc = nn.Linear(512, 31)

    def forward(self, x):
        logits = self.model(x)
        return logits
model = ResNet18().to(device)
model.load_state_dict(torch.load('model/resNet/ResNet_2.pt', map_location=device))

NameError: name 'nn' is not defined

In [29]:
import os, cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
path = 'test/photo/2.PNG'
img = cv2.imread(path)
copy = cv2.resize(img, (64, 64))
transform = transforms.ToTensor()
image = transform(copy)

with torch.no_grad():
    image = torch.unsqueeze(image, 0)
    image = image.to(device)
    prediction = model(image)
    predicted_class = prediction.argmax()
    print(prediction)
    print(predicted_class.item())

tensor([[-0.4376,  0.7432,  0.2836, -0.5066, -0.1940, -0.0799, -0.4370,  0.0272,
          0.8172, -0.0355, -0.2009, -0.1893, -0.0667,  0.1740,  1.2434, -0.2515,
         -0.3809,  0.0077, -0.3301, -0.5201,  0.9397,  0.0565,  0.8155, -0.3965,
         -0.1972,  0.0583, -0.7525, -0.3545,  0.0101, -0.1634, -0.4118]])
14
